# DataFrame com Dados dos Filmes (movies)

In [2]:
colunas_info_movie = ['id_hashed', 'movie_title', 'color', 'duration', 'genres', 'plot_keywords', 'language', 'country', 'content_rating', 'title_year', 'aspect_ratio','movie_imdb_link']

In [3]:
df.select(colunas_info_movie).write.format('parquet')\
.option("header","true")\
.mode('overwrite')\
.save("dbfs:/FileStore/trusted/info_movie")

NameError: name 'df' is not defined

In [ ]:
df_info_movie = spark.read.parquet("dbfs:/FileStore/trusted/info_movie")
df_info_movie.createOrReplaceTempView('df_info_movie')
display(df_info_movie)

### Analise Matheus

In [ ]:
%sql
-- selecionando filmes do gênero 
select movie_title, genres from df_info_movie
where genres like '%Comedy%'

In [ ]:
%sql
-- selecionando filmes de 2015 para frente
select movie_title, title_year from df_info_movie
where title_year > 2015